<a href="https://colab.research.google.com/github/uygishypr/Technical_Project/blob/master/Colab_August_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import mean, nanargmin
from numpy import std
from matplotlib import pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import cv


In [4]:
df_merged_sorted = pd.read_excel("Nans_removed_dataset.xlsx", index_col =  [0, 1])
companies_list = df_merged_sorted.index.get_level_values('Company Name').unique().tolist()
print(companies_list)

['1-800-Flowers.Com Inc', '11 88 0 Solutions AG', '111 Inc', '17 Education & Technology Group Inc', '22nd Century Group Inc', '2G Energy AG', '2U Inc', '3M Co', '3P Learning Ltd', '5N Plus Inc', '888 Holdings PLC', 'A G Barr PLC', 'A O Smith Corp', 'A and W Revenue Royalties Income Fund', 'A2 Milk Company Ltd', 'A2B Australia Ltd', 'AAC Clyde Space AB', 'AAK AB (publ)', 'AAPICO Hitech PCL', 'AAR Corp', 'ABC Technologies Holdings Inc', 'ABC-Mart Inc', 'ABM Industries Inc', 'ACC Ltd', 'ACCO Brands Corp', 'ACS Actividades de Construccion y Servicios SA', 'ADAMA Ltd', 'ADT Inc', 'AECC Aviation Power Co Ltd', 'AECI Ltd', 'AECOM', 'AG Anadolu Grubu Holding AS', 'AGC Inc', 'AGCO Corp', 'AIA Engineering Ltd', 'AIC Mines Ltd', 'AK Alrosa PAO', 'AKR Corporindo Tbk PT', 'ALAFCO Avaiation Lease and Finance Co KSCP', 'ALD SA', 'ALS Ltd', 'AMA Group Ltd', 'AMETEK Inc', 'AMG Advanced Metallurgical Group NV', 'ANA Holdings Inc', 'ANTA Sports Products Ltd', 'AO World PLC', 'AP Moeller - Maersk A/S', 'A

In [5]:
# Check the number of missing values before filling
df_merged_sorted.isna().sum()
columns_list = list(df_merged_sorted.drop(columns=['Sum','Label','Identifier (RIC)']))
df_list = []

In [6]:
for company in companies_list:

    subset_df = df_merged_sorted.loc[company].bfill(axis = "rows")
    df_list.append(subset_df)

vertical_concat = pd.concat(df_list, axis=0)
vertical_concat.isna().sum() 

GICS Sector Name                                         0
Identifier (RIC)                                         0
Health&SafetyPolicy                                      2
PolicyEmployeeHealth&Safety                            307
PolicySupplyChainHealth&Safety                        1495
TrainingandDevelopmentPolicy                             2
PolicySkillsTraining                                   470
PolicyCareerDevelopment                                579
PolicyDiversityandOpportunity                          636
TargetsDiversityandOpportunity                        1760
EmployeesHealth&SafetyTeam                            1142
Health&SafetyTraining                                  636
SupplyChainHealth&SafetyTraining                      1753
SupplyChainHealth&SafetyImprovements                  1827
EmployeesHealth&SafetyOHSAS18001                      1058
FlexibleWorkingHours                                  1547
DayCareServices                                       17

In [6]:
vertical_concat.columns

Index(['GICS Sector Name', 'Identifier (RIC)', 'Health&SafetyPolicy',
       'PolicyEmployeeHealth&Safety', 'PolicySupplyChainHealth&Safety',
       'TrainingandDevelopmentPolicy', 'PolicySkillsTraining',
       'PolicyCareerDevelopment', 'PolicyDiversityandOpportunity',
       'TargetsDiversityandOpportunity', 'EmployeesHealth&SafetyTeam',
       'Health&SafetyTraining', 'SupplyChainHealth&SafetyTraining',
       'SupplyChainHealth&SafetyImprovements',
       'EmployeesHealth&SafetyOHSAS18001', 'FlexibleWorkingHours',
       'DayCareServices', 'InternalPromotion', 'ManagementTraining',
       'SupplierESGtraining', 'EmployeeResourceGroups', 'HumanRightsPolicy',
       'PolicyFreedomofAssociation', 'PolicyChildLabor', 'PolicyForcedLabor',
       'PolicyHumanRights', 'FundamentalHumanRightsILOUN',
       'HumanRightsContractor', 'EthicalTradingInitiativeETI',
       'HumanRightsBreachesContractor', 'PolicyFairCompetition',
       'PolicyBriberyandCorruption', 'PolicyBusinessEthics',
   

In [7]:
vertical_concat_features = vertical_concat.iloc[:,0:56]

vertical_concat_features.drop(columns = ["Identifier (RIC)"],inplace = True)

In [8]:
vertical_concat_features.columns

Index(['GICS Sector Name', 'Health&SafetyPolicy',
       'PolicyEmployeeHealth&Safety', 'PolicySupplyChainHealth&Safety',
       'TrainingandDevelopmentPolicy', 'PolicySkillsTraining',
       'PolicyCareerDevelopment', 'PolicyDiversityandOpportunity',
       'TargetsDiversityandOpportunity', 'EmployeesHealth&SafetyTeam',
       'Health&SafetyTraining', 'SupplyChainHealth&SafetyTraining',
       'SupplyChainHealth&SafetyImprovements',
       'EmployeesHealth&SafetyOHSAS18001', 'FlexibleWorkingHours',
       'DayCareServices', 'InternalPromotion', 'ManagementTraining',
       'SupplierESGtraining', 'EmployeeResourceGroups', 'HumanRightsPolicy',
       'PolicyFreedomofAssociation', 'PolicyChildLabor', 'PolicyForcedLabor',
       'PolicyHumanRights', 'FundamentalHumanRightsILOUN',
       'HumanRightsContractor', 'EthicalTradingInitiativeETI',
       'HumanRightsBreachesContractor', 'PolicyFairCompetition',
       'PolicyBriberyandCorruption', 'PolicyBusinessEthics',
       'PolicyCommunity

In [ ]:
# N_neighbors is a preprocessing hyperparameter
impute_knn = KNNImputer(n_neighbors=9)
knn_imputed_df = impute_knn.fit_transform(vertical_concat_features)

In [9]:
ohe = OneHotEncoder(sparse=False)
ohe_transformed = ohe.fit_transform(vertical_concat_features)

In [10]:
#%% TRAINING
# Drop the row that has NaN values (last row)
X = np.array(vertical_concat_features)
y = np.array(vertical_concat["Label"])
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.2, random_state=42)


In [11]:
X_ohe = np.array(ohe_transformed)
y_ohe = np.array(vertical_concat["Label"])
X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(X_ohe, y_ohe, 
                                   test_size=0.2,random_state=42, stratify=y_ohe)


In [ ]:
#%%
results = list()
strategies = [str(i) for i in [1,3,5,7,9,15,18,21]]

for s in strategies:
	# create the modeling pipeline
    pipeline = Pipeline(steps=[('i', KNNImputer(n_neighbors=int(s))), ('m', RandomForestClassifier())])
    # define model evaluation
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(pipeline, X, y, scoring='precision', cv=cv, n_jobs=-1, error_score='raise')
    results.append(scores)
    print('Mean Precision: %.3f (%.3f)' % (mean(scores), std(scores)))

pyplot.boxplot(results, labels=strategies, showmeans=True)
pyplot.show()


In [ ]:
#%%
from xgboost import XGBClassifier

xgboost_model = XGBClassifier()
xgboost_model.fit(X_train, y_train)

y_pred = xgboost_model.predict(X_test)

predictions = [round(value) for value in y_pred]

precision = precision_score(y_test, predictions)


In [ ]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train,y_train)
predictions = tree_clf.predict(X_test)

rnd_clf = RandomForestClassifier(n_estimators=500)
rnd_clf.fit(X_train,y_train)
rf_predictions = rnd_clf.predict(X_test)

print("Random Forest Accuracy: ", accuracy_score(y_test, rf_predictions))
print("Random Forest Precision: ", precision_score(y_test, rf_predictions))
print("Random Forest Recall: ", recall_score(y_test, rf_predictions))
confusion_matrix(y_test,rf_predictions)

print(accuracy_score(y_test, predictions))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
confusion_matrix(y_test,predictions)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42) 
X_res, y_res = sm.fit_resample(X_train, y_train)

print('Original dataset shape %s'%Counter(y_train))
print('Resampled dataset shape %s'%Counter(y_res))

# Test the newly populated dataset 
rf_clf_resampled = RandomForestClassifier()
rf_clf_resampled.fit(X_res,y_res)
rf_predictions_after_resampled = rf_clf_resampled.predict(X_test)

print(confusion_matrix(y_test, rf_predictions_after_resampled))
print(accuracy_score(y_test, rf_predictions_after_resampled))
print(precision_score(y_test, rf_predictions_after_resampled))
print(recall_score(y_test, rf_predictions_after_resampled))


In [15]:
X_ohe.shape

(28648, 166)

In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13065003737839013090
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16139419648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 854035294277164809
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]


In [13]:
import tensorflow as tf
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



In [14]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [17]:
encoder = LabelEncoder()
encoder.fit(y_ohe)
encoded_Y = encoder.transform(y_ohe)

In [21]:
from sklearn.preprocessing import LabelEncoder
from keras import metrics

with tf.device("/device:GPU:0"):

  # create model
	model = Sequential()
	model.add(Dense(166, input_dim=166, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.AUC(),
																																	metrics.Precision(),
																																	metrics.Recall()]) 
 

	history = model.fit(
		X_train_ohe,
    y_train_ohe,
    batch_size=128,
    epochs=100,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_test_ohe, y_test_ohe),
)



Epoch 1/100
180/180 [==============================] - 4s 15ms/step - loss: 0.3423 - auc_4: 0.7310 - precision_4: 0.6241 - recall_4: 0.0581 - val_loss: 0.3227 - val_auc_4: 0.7743 - val_precision_4: 0.7164 - val_recall_4: 0.1304
Epoch 2/100
180/180 [==============================] - 2s 9ms/step - loss: 0.3249 - auc_4: 0.7672 - precision_4: 0.6892 - recall_4: 0.1107 - val_loss: 0.3180 - val_auc_4: 0.7802 - val_precision_4: 0.7516 - val_recall_4: 0.1603
Epoch 3/100
180/180 [==============================] - 1s 7ms/step - loss: 0.3220 - auc_4: 0.7726 - precision_4: 0.6661 - recall_4: 0.1260 - val_loss: 0.3169 - val_auc_4: 0.7808 - val_precision_4: 0.6368 - val_recall_4: 0.1929
Epoch 4/100
180/180 [==============================] - 1s 7ms/step - loss: 0.3185 - auc_4: 0.7794 - precision_4: 0.6636 - recall_4: 0.1447 - val_loss: 0.3149 - val_auc_4: 0.7868 - val_precision_4: 0.6979 - val_recall_4: 0.1821
Epoch 5/100
180/180 [==============================] - 1s 7ms/step - loss: 0.3170 - auc_4: 

In [ ]:
	estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=2)
	kfold = StratifiedKFold(n_splits=10, shuffle=True)
	results = cross_val_score(estimator, X_ohe, encoded_Y, cv=kfold)